In [1]:
import sys, os
import pandas as pd
import numpy as np

# Ensure project root is on path
sys.path.append("/Users/lei/Documents/Sequenzo_all_folders/glm_plus")

from glm_plus.frequentist.torque import FrequentistOQR

In [2]:
df_seniority = pd.read_csv("/Users/lei/Documents/Sequenzo_all_folders/sequenzo_local/test_data/real_data_my_paper/detailed_senority_levels_for_10_years.csv")
df_seniority

,worker_id,gender,country,cohort,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,work_years,company_size,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,dilip-kumar-nath-7538746a,male,india,11-20 total work years,Bachelor,False,Multinational,15.250,"1,001-5,000 employees",Regular,Regular,Regular,Regular,Senior,Senior,Senior,Senior,Senior,Senior
1,vinodkumar-yadav-b7a976107,male,india,11-20 total work years,Bachelor,False,Local,11.500,"5,001-10,000 employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
2,rajani-kulkarni-72674649,female,india,11-20 total work years,Bachelor,False,Multinational,11.250,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Leader,Leader,Regular
3,josh-slosson-07a1509,male,india,11-20 total work years,Bachelor,False,International,18.417,11-50 employees,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
4,jonatthan-rodriguez-8a5612121,male,india,11-20 total work years,Bachelor,False,Multinational,11.917,"5,001-10,000 employees",Regular,Regular,Senior,Senior,Senior,Senior,Senior,Senior,Senior,Senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32623,edwardgu1915,male,us,0-10 total work years,Master,False,Multinational,10.499,"501-1,000 employees",Leader,Leader,Leader,Leader,Leader,Senior,Senior,Senior,Regular,Senior
32624,terry-ferguson-8060363a,male,us,21-30 total work years,Bachelor,False,Local,21.001,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
32625,christy-garner-677894213,female,us,21-30 total work years,Bachelor,False,Multinational,27.250,"501-1,000 employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
32626,jay-p-119369202,male,us,21-30 total work years,Bachelor,False,Local,23.417,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular


In [3]:
# Wide-format cleaning: collapse seniority labels into 4 levels
# - Assistant + Junior → Assistant/Junior
# - Regular → Regular
# - Senior/Leader/Lead → Leader
# - Chief or Founder (+ VP/Director/C-level variants) → Chief/Founder

# Detect Y columns
Y_cols = [c for c in df_seniority.columns if str(c).upper().startswith('Y')]
assert len(Y_cols) > 0, 'No Y1..Yk columns found'

collapse_order = ["Assistant/Junior", "Regular", "Leader", "Chief/Founder"]

def _collapse_label(v):
    if pd.isna(v):
        return np.nan
    t = str(v).strip().lower()
    if t in {"assistant", "intern", "entry level"}:
        return "Assistant/Junior"
    if t in {"junior", "jr"}:
        return "Assistant/Junior"
    if t in {"regular", "mid", "middle", "intermediate", "associate"}:
        return "Regular"
    if t in {"senior", "leader", "lead", "sr"}:
        return "Leader"
    if t in {"chief or founder", "chief/founder", "chief", "c-level", "founder", "executive", "vp", "vice president", "director"}:
        return "Chief/Founder"
    # keep original if already one of target labels
    if v in collapse_order:
        return v
    return v

for col in Y_cols:
    df_seniority[col] = df_seniority[col].apply(_collapse_label)

# Quick check of levels after collapsing
level_counts = pd.DataFrame({c: df_seniority[c].value_counts(dropna=False) for c in Y_cols})
print(level_counts.sum(axis=1).sort_index())

df_seniority.head()


Assistant/Junior     11110
Chief/Founder         2504
Leader              114640
Regular             198026
dtype: int64


,worker_id,gender,country,cohort,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,work_years,company_size,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,dilip-kumar-nath-7538746a,male,india,11-20 total work years,Bachelor,False,Multinational,15.250,"1,001-5,000 employees",Regular,Regular,Regular,Regular,Leader,Leader,Leader,Leader,Leader,Leader
1,vinodkumar-yadav-b7a976107,male,india,11-20 total work years,Bachelor,False,Local,11.500,"5,001-10,000 employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
2,rajani-kulkarni-72674649,female,india,11-20 total work years,Bachelor,False,Multinational,11.250,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Leader,Leader,Regular
3,josh-slosson-07a1509,male,india,11-20 total work years,Bachelor,False,International,18.417,11-50 employees,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
4,jonatthan-rodriguez-8a5612121,male,india,11-20 total work years,Bachelor,False,Multinational,11.917,"5,001-10,000 employees",Regular,Regular,Leader,Leader,Leader,Leader,Leader,Leader,Leader,Leader


In [ ]:
# 横截面：以 Y10 为因变量（有序），gender 为核心自变量，控制：
# highest_educational_degree, whether_bachelor_university_prestigious,
# internationalization, work_years, company_size

year_col = 'Y10'
order_labels = ["Assistant/Junior", "Regular", "Leader", "Chief/Founder"]

use_cols = [
    'gender',
    'highest_educational_degree',
    'whether_bachelor_university_prestigious',
    'internationalization',
    'work_years',
    'company_size',
    year_col,
]

df_cs_full = df_seniority[use_cols].dropna().copy()

# y: 映射到 1..J
cat_type = pd.CategoricalDtype(categories=order_labels, ordered=True)
df_cs_full['y_cat'] = df_cs_full[year_col].astype(cat_type)
df_cs_full['y'] = df_cs_full['y_cat'].cat.codes + 1

# 性别：female=1, male=0
_df_gender = df_cs_full['gender'].astype(str).str.strip().str.lower()
df_cs_full['female'] = (_df_gender == 'female').astype(int)

# 是否名校本科学位：转为 0/1
col_prestige = 'whether_bachelor_university_prestigious'
if str(df_cs_full[col_prestige].dtype) == 'bool':
    df_cs_full['prestigious_bachelor'] = df_cs_full[col_prestige].astype(int)
else:
    df_cs_full['prestigious_bachelor'] = (
        df_cs_full[col_prestige].astype(str).str.strip().str.lower().isin(['true','1','yes','y','t'])
    ).astype(int)

# 数值特征
df_cs_full['work_years'] = pd.to_numeric(df_cs_full['work_years'], errors='coerce')

# 类别特征 one-hot（drop_first 保留基准类）
cat_cols = ['highest_educational_degree', 'internationalization', 'company_size']
X_cat = pd.get_dummies(df_cs_full[cat_cols], columns=cat_cols, drop_first=True, dtype=int)

# 组合设计矩阵：先放 gender，再放控制
X_df = pd.concat([
    df_cs_full[['female', 'prestigious_bachelor', 'work_years']].reset_index(drop=True),
    X_cat.reset_index(drop=True)
], axis=1)

feature_cols = list(X_df.columns)
X = X_df.to_numpy(dtype=float)
y = df_cs_full['y'].to_numpy(dtype=int)

len(df_cs_full), X.shape, y.shape, np.unique(y)


In [ ]:
# 拟合 TORQUE 频率学 OQR（横截面）
model = FrequentistOQR(
    quantiles=(0.25, 0.5, 0.75),
    use_two_index=True,
    auto_select_k=True,
    subsample_n=5000 if len(df_cs_full) > 20000 else (3000 if len(df_cs_full) > 2000 else None),
    random_state=0,
)
model.fit(X, y)

info = model.summary()
info['selected_k'], info['selected_k_full'], info.get('corr_xbeta1_xbeta2', None)


In [ ]:
# 预测区间与覆盖率评估（按 Table 6 风格）
lo, hi = model.predict_interval(X, tau_low=0.25, tau_high=0.75)
res = model.evaluate_intervals(X, y, tau_low=0.25, tau_high=0.75)

print({'mean_length': res['mean_length'], 'mean_coverage': res['mean_coverage']})
print('length_hist (first 6):', res['length_hist'][:6])
res['coverage_by_length']


In [ ]:
# 报告用：把某个变量系数重标定为 1（只用于展示，不改模型）
# 例如把第 0 个特征的 beta1 重标定
report = model.get_reporting_scaled_betas(feature_index=0)
report['beta1_scaled'][:5] if report['beta1_scaled'] is not None else None


In [ ]:
# 粘地板 / 玻璃天花板：分位点上的性别差异剖面
# 思路：在控制变量固定的情况下，对 female=0/1 的预测中位数（或多个分位点）做对比，
# 再按“底部 vs 顶部”的区间计算差值是否更大。

import numpy as np
import pandas as pd

# 构造一个代表性的控制变量参考点：连续变量取中位数，类别取众数
X_ref = X_df.copy()
X_ref['female'] = 0
X_ref_m = X_ref.median(numeric_only=True)
X_ref = X_ref.copy()
for c in X_df.columns:
    if c == 'female':
        continue
    if X_df[c].dtype.kind in 'biufc':
        X_ref[c] = X_df[c].median()
    else:
        X_ref[c] = X_df[c].mode().iat[0] if not X_df[c].mode().empty else X_df[c].iloc[0]

# 生成 female=0/1 的预测分位数（连续尺度）
quantiles = (0.1, 0.25, 0.5, 0.75, 0.9)
X0 = X_ref.copy(); X0['female'] = 0
X1 = X_ref.copy(); X1['female'] = 1

pred0_cont = model.predict_quantiles_continuous(X0.to_numpy(dtype=float), quantiles=quantiles)
pred1_cont = model.predict_quantiles_continuous(X1.to_numpy(dtype=float), quantiles=quantiles)

# 计算每个分位点的性别差：male - female（正值表示女性处境更差，即更难上移）
# 注意：这里用连续尺度，便于比较“跨级难度”。
delta_by_tau = {tau: float(np.mean(pred0_cont[tau] - pred1_cont[tau])) for tau in quantiles}

# 定义“底部”和“顶部”两段：
bottom_taus = [0.1, 0.25]
top_taus = [0.75, 0.9]

bottom_gap = float(np.mean([delta_by_tau[t] for t in bottom_taus]))
top_gap = float(np.mean([delta_by_tau[t] for t in top_taus]))

conclusion = (
    'sticky floor' if bottom_gap > top_gap + 1e-6 else (
    'glass ceiling' if top_gap > bottom_gap + 1e-6 else 'mixed/neutral')
)

print({'bottom_gap(male-female)': bottom_gap, 'top_gap(male-female)': top_gap, 'conclusion': conclusion})
delta_by_tau


In [ ]:
# 可视化：各分位点的性别差（male-female），底部/顶部区间对比
import matplotlib.pyplot as plt

qs = sorted(delta_by_tau.keys())
vals = [delta_by_tau[q] for q in qs]

plt.figure(figsize=(5.5,3.5))
plt.plot(qs, vals, marker='o')
plt.axhline(0, color='gray', lw=1)
plt.axvspan(0.0, 0.3, color='tab:blue', alpha=0.08, label='bottom zone')
plt.axvspan(0.7, 1.0, color='tab:red', alpha=0.08, label='top zone')
plt.title('Gender gap profile across quantiles (male - female)')
plt.xlabel('Quantile')
plt.ylabel('Gap on continuous scale')
plt.legend()
plt.show()


In [ ]:
# 稳健性：用不同控制向量的代表点，以及用预测“区间边界”来度量顶部/底部差异
# 方案A：改变代表点（把连续变量换成四分位点、类别换成其它基类），看结论是否稳定
# 方案B：使用预测区间边界的 male-female 差值（lo/hi），与连续分位点的结论是否一致

from itertools import product

control_summaries = {
    'work_years': [X_df['work_years'].quantile(0.25), X_df['work_years'].median(), X_df['work_years'].quantile(0.75)]
}

# 类别基类切换：把 one-hot 的 1 位置挪到别的列上（通过构造一个行向量）
def make_ref_row(female_val: int, base_choices: dict[str, str]):
    row = {c: 0.0 for c in X_df.columns}
    row['female'] = float(female_val)
    row['prestigious_bachelor'] = float(X_df['prestigious_bachelor'].median())
    row['work_years'] = float(base_choices.get('work_years', X_df['work_years'].median()))
    # 对每个类别组，找以该名称开头的列，保持全 0 即表示基准类
    for c in X_df.columns:
        if c.startswith('highest_educational_degree_'):
            pass
        if c.startswith('internationalization_'):
            pass
        if c.startswith('company_size_'):
            pass
    return pd.DataFrame([row])

# 取几个代表点
ref_rows_0 = [make_ref_row(0, {'work_years': wy}) for wy in control_summaries['work_years']]
ref_rows_1 = [make_ref_row(1, {'work_years': wy}) for wy in control_summaries['work_years']]

qs = (0.1, 0.25, 0.5, 0.75, 0.9)
gap_sets = []
for r0, r1 in zip(ref_rows_0, ref_rows_1):
    pred0 = model.predict_quantiles_continuous(r0.to_numpy(dtype=float), quantiles=qs)
    pred1 = model.predict_quantiles_continuous(r1.to_numpy(dtype=float), quantiles=qs)
    delta = {t: float(pred0[t] - pred1[t]) for t in qs}
    gap_sets.append(delta)

robust_bottom = np.mean([np.mean([g[0.1], g[0.25]]) for g in gap_sets])
robust_top = np.mean([np.mean([g[0.75], g[0.9]]) for g in gap_sets])
print({'robust_bottom': robust_bottom, 'robust_top': robust_top})
'consistent with ' + ('sticky floor' if robust_bottom > robust_top else 'glass ceiling' if robust_top > robust_bottom else 'mixed/neutral')


In [ ]:
# 按国家分组评估“粘地板/玻璃天花板”
# 构建子集设计矩阵、拟合模型、计算 bottom/top gaps 与结论

from typing import Dict, Tuple

order_labels = ["Assistant/Junior", "Regular", "Leader", "Chief/Founder"]

def build_design_for_subset(df_sub: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, np.ndarray, np.ndarray]:
    cols_needed = [
        'gender', 'highest_educational_degree',
        'whether_bachelor_university_prestigious',
        'internationalization', 'work_years', 'company_size', 'Y10'
    ]
    d = df_sub[cols_needed].dropna().copy()
    # y
    cat_type = pd.CategoricalDtype(categories=order_labels, ordered=True)
    d['y_cat'] = d['Y10'].astype(cat_type)
    d['y'] = d['y_cat'].cat.codes + 1
    # female
    gg = d['gender'].astype(str).str.strip().str.lower()
    d['female'] = (gg == 'female').astype(int)
    # prestige
    col_p = 'whether_bachelor_university_prestigious'
    if str(d[col_p].dtype) == 'bool':
        d['prestigious_bachelor'] = d[col_p].astype(int)
    else:
        d['prestigious_bachelor'] = (
            d[col_p].astype(str).str.strip().str.lower().isin(['true','1','yes','y','t'])
        ).astype(int)
    # numeric
    d['work_years'] = pd.to_numeric(d['work_years'], errors='coerce')
    d = d.dropna(subset=['work_years']).copy()
    # one-hot
    cat_cols = ['highest_educational_degree', 'internationalization', 'company_size']
    X_cat = pd.get_dummies(d[cat_cols], columns=cat_cols, drop_first=True, dtype=int)
    X_df = pd.concat([
        d[['female', 'prestigious_bachelor', 'work_years']].reset_index(drop=True),
        X_cat.reset_index(drop=True)
    ], axis=1)
    X = X_df.to_numpy(dtype=float)
    y = d['y'].to_numpy(dtype=int)
    return d, X_df, X, y


def compute_gender_gap_profile(model: FrequentistOQR, X_df_in: pd.DataFrame,
                               quantiles=(0.1,0.25,0.5,0.75,0.9)) -> Tuple[Dict[float, float], float, float, str]:
    # 构造代表点（基准类 + 中位数控制），只改 female
    ref = {c: 0.0 for c in X_df_in.columns}
    ref['female'] = 0.0
    if 'prestigious_bachelor' in X_df_in:
        ref['prestigious_bachelor'] = float(X_df_in['prestigious_bachelor'].median())
    if 'work_years' in X_df_in:
        ref['work_years'] = float(X_df_in['work_years'].median())
    X0 = pd.DataFrame([ref])
    X1 = pd.DataFrame([ref]); X1['female'] = 1.0
    pred0 = model.predict_quantiles_continuous(X0.to_numpy(dtype=float), quantiles=quantiles)
    pred1 = model.predict_quantiles_continuous(X1.to_numpy(dtype=float), quantiles=quantiles)
    delta = {t: float(pred0[t] - pred1[t]) for t in quantiles}
    bottom_taus, top_taus = [0.1, 0.25], [0.75, 0.9]
    bottom_gap = float(np.mean([delta[t] for t in bottom_taus]))
    top_gap = float(np.mean([delta[t] for t in top_taus]))
    concl = 'sticky floor' if bottom_gap > top_gap + 1e-6 else ('glass ceiling' if top_gap > bottom_gap + 1e-6 else 'mixed/neutral')
    return delta, bottom_gap, top_gap, concl


def evaluate_by_country(min_n: int = 500) -> pd.DataFrame:
    rows = []
    for ctry, df_g in df_seniority.groupby('country'):
        d, Xdf, Xg, yg = build_design_for_subset(df_g)
        n = len(yg)
        if n < min_n:
            continue
        m = FrequentistOQR(
            quantiles=(0.25, 0.5, 0.75),
            use_two_index=True,
            auto_select_k=True,
            subsample_n=5000 if n > 20000 else (3000 if n > 2000 else None),
            random_state=0,
        )
        m.fit(Xg, yg)
        delta, bottom_gap, top_gap, concl = compute_gender_gap_profile(m, Xdf)
        rows.append({
            'country': str(ctry),
            'n': n,
            'bottom_gap': bottom_gap,
            'top_gap': top_gap,
            'gap_diff_bottom_minus_top': float(bottom_gap - top_gap),
            'conclusion': concl,
        })
    out = pd.DataFrame(rows).sort_values(['conclusion','gap_diff_bottom_minus_top'], ascending=[True, False])
    return out


In [ ]:
# 运行国家分组评估（可调最小样本量门槛）
country_summary = evaluate_by_country(min_n=500)
country_summary.head(20)
